In [14]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50,MobileNetV2,MobileNet,InceptionV3
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.utils import to_categorical 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os

In [15]:
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [16]:
args = {
	"dataset": "data",
	"model": "model/violence.h5",
    "label-bin": "model/lb.pickle",
    "epochs": 10,
    "plot": "plot.png"
    
}


In [17]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Violence", "NonViolence"])

In [18]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('-'*100)
print("[INFO] loading images...")
print('-'*100)
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []


----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------


In [19]:
print(imagePaths)

['data\\NonViolence\\free-hug-0180.jpg', 'data\\NonViolence\\free-hug-0185.jpg', 'data\\NonViolence\\free-hug-0190.jpg', 'data\\NonViolence\\free-hug-0195.jpg', 'data\\NonViolence\\free-hug-0200.jpg', 'data\\NonViolence\\free-hug-0205.jpg', 'data\\NonViolence\\free-hug-0210.jpg', 'data\\NonViolence\\free-hug-0215.jpg', 'data\\NonViolence\\free-hug-0220.jpg', 'data\\NonViolence\\free-hug-0225.jpg', 'data\\NonViolence\\free-hug-0230.jpg', 'data\\NonViolence\\free-hug-0235.jpg', 'data\\NonViolence\\free-hug-0240.jpg', 'data\\NonViolence\\free-hug-0245.jpg', 'data\\NonViolence\\free-hug-0250.jpg', 'data\\NonViolence\\free-hug-0255.jpg', 'data\\NonViolence\\free-hug-0260.jpg', 'data\\NonViolence\\free-hug-0265.jpg', 'data\\NonViolence\\free-hug-0270.jpg', 'data\\NonViolence\\free-hug-0275.jpg', 'data\\NonViolence\\free-hug-0280.jpg', 'data\\NonViolence\\free-hug-0285.jpg', 'data\\NonViolence\\free-hug-0290.jpg', 'data\\NonViolence\\free-hug-0295.jpg', 'data\\NonViolence\\free-hug-0300.jpg',

In [20]:
# loop over the image paths
for imagePath in tqdm(imagePaths[::2]):
	# imagePath : file name ex) V_123
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2] # Violence / NonViolence

	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	if label not in LABELS:
		continue

	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

100%|█████████████████████████████████████████████████████████████████████████████| 8790/8790 [01:12<00:00, 121.21it/s]


In [21]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

In [22]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [23]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [24]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [25]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [26]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [27]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = InceptionV3(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [28]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

In [29]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [30]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
model.trainable = True
# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [31]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(model.summary())

----------------------------------------------------------------------------------------------------
[INFO] compiling model...
----------------------------------------------------------------------------------------------------
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
_________________________________________________________

In [33]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=args["epochs"])



----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
206/206 [==============================] - 3865s 19s/step - loss: 0.5841 - accuracy: 0.6881 - val_loss: 0.3479 - val_accuracy: 0.9003
Epoch 2/10
206/206 [==============================] - 3602s 17s/step - loss: 0.3395 - accuracy: 0.8665 - val_loss: 0.2253 - val_accuracy: 0.9416
Epoch 3/10
206/206 [==============================] - 3673s 18s/step - loss: 0.2238 - accuracy: 0.9204 - val_loss: 0.1460 - val_accuracy: 0.9632
Epoch 4/10
206/206 [==============================] - 3790s 18s/step - loss: 0.1709 - accuracy: 0.9367 - val_loss: 0.1107 - val_accuracy: 0.9688
Epoch 5/10
206/206 [==============================] - 3528s 17s/step - loss: 0.1338 - accuracy: 0.9552 - val_loss: 0.0

In [34]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

----------------------------------------------------------------------------------------------------
[INFO] evaluating network...
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

 NonViolence       0.98      1.00      0.99       935
    Violence       1.00      0.98      0.99      1263

    accuracy                           0.99      2198
   macro avg       0.99      0.99      0.99      2198
weighted avg       0.99      0.99      0.99      2198



In [35]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

----------------------------------------------------------------------------------------------------
[INFO] plot the training loss and accuracy...
----------------------------------------------------------------------------------------------------


In [36]:
# serialize the model to disk
print('-'*100)
print("[INFO] serializing network...")
print('-'*100)
model.save(args["model"])

----------------------------------------------------------------------------------------------------
[INFO] serializing network...
----------------------------------------------------------------------------------------------------


In [37]:
# serialize the label binarizer to disk
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()